### Importing important libraries


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#import tensorflow as tf
import keras
from keras.layers import Dense, CuDNNLSTM, Dropout, Embedding, SpatialDropout1D, Recurrent
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import Sequential
from keras.utils import to_categorical
#import re

Using TensorFlow backend.


### Importing the dataset


In [0]:
df = pd.read_csv('train.csv')
df.drop(columns=['id'], inplace=True)

In [5]:
df.head()

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


### Text Preprocessing
1. we have to convert all the text to lower case
2. remove all the punctuations and digits
3. we tokenize the sentences using the Keras Tokenizer
4. after tokenizing we convert the sentences into sequence
5. then pad the sequence (by default it is "pre" padding)


In [0]:
df['title'] = df.title.str.lower()    #converting all text to lower case
df['author'] = df.author.str.lower()
df['text'] = df.text.astype('str')
df['text'] = df.text.str.lower()

In [0]:
#df['text'] = df['text'].apply((lambda x : re.sub('[^a-zA-Z]', ' ', x)))   #removing all punctuatations and digits

In [0]:
max_features = 5000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

In [0]:
embed_dim = 128
lstm_out = 196

### Train & validation set splitting

In [0]:
Y = to_categorical(df['label'].values)
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y)

In [0]:
del X

### Model creation

In [0]:
model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(CuDNNLSTM(lstm_out))
model.add(Dropout(0.5))

model.add(Dense(2, activation="softmax"))
model.compile(optimizer="adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [0]:
X_train1 = X_train[8000:9000]
Y_train1 = Y_train[8000:9000]     #used

In [42]:
model.fit(X_train1, Y_train1, epochs=1, batch_size=16)

Epoch 1/1
1000/1000 [==============================] - 227s 227ms/step - loss: 0.3527 - acc: 0.8470


In [47]:
model.evaluate(X_val, Y_val, batch_size=16)

5200/5200 [==============================] - 409s 79ms/step


[0.2562533688315978, 0.9026923076923077]

In [0]:
model.save('fake_news.h5')